In [22]:
import pandas as pd
import numpy as np

In [2]:
items = pd.read_csv('items.csv')

In [4]:
ratings = pd.read_csv('ratings.csv')

In [6]:
cbf = pd.read_csv('cbf.csv')

In [8]:
item_item = pd.read_csv('item-item.csv')

In [10]:
user_user = pd.read_csv('user-user.csv')

In [11]:
pers_bias = pd.read_csv('pers-bias.csv')

In [12]:
mf = pd.read_csv('mf.csv')

In [28]:
recs = [cbf, item_item, user_user, pers_bias, mf]
recs_names = ['cbf', 'item_item', 'user_user', 'pers_bias', 'mf']

# Helper functions

In [17]:
def get_ratings(user_id):
    user_ratings = ratings[user_id]
    actual_ratings = user_ratings[~np.isnan(user_ratings)]
    return actual_ratings

In [106]:
def get_top_n(user_id, n):
    top_n = {}
    for rec, rec_name in zip(recs, recs_names):
        top_n_items = rec[user_id].argsort().sort_values()[:n].index.values
        top_n[rec_name] = top_n_items
    return top_n

In [137]:
def get_popular_items(n):
    pop_percentages = ratings.copy()
    pop_percentages['popularity'] = ratings.apply(lambda row: np.sum(~np.isnan(row))-1, axis=1)/len(ratings.columns[1::])
    pop_percentages = pop_percentages.sort_values(by = 'popularity', ascending=False)
    return pop_percentages.item.values[:n]

# RMSE

In [32]:
def get_rmse(user_id):   
    user_ratings = get_ratings(user_id)
    rmse = {}
    for rec, rec_name in zip(recs, recs_names):
        predicted_ratings = rec.loc[user_ratings.index, user_id]
        temp = np.sqrt(np.average((predicted_ratings - user_ratings)**2))
        rmse[rec_name] = temp
    return rmse

# Precision @ N

In [39]:
def get_precision_at_n(user_id, n):
    top_n = get_top_n(user_id, n)
    user_ratings = get_ratings(user_id).index.values
    precisions = {}
    for rec, rec_name in zip(recs, recs_names):
        temp = np.sum(np.isin(top_n[rec_name], user_ratings))/n
        precisions[rec_name] = temp
    return precisions

# Product Diversity

In [76]:
# We will use the "FullCat" column in the items catalog to determine the product diversity in the recommendations.
# The recommender with a high number of distinct product categories in its recommendations is said to be product-diverse
def get_product_diversity(user_id, n):
    top_n = get_top_n(user_id, n)
    product_diversity = {}
    for rec_name in top_n:
        categories = items.loc[top_n[rec_name]][['FullCat']].values
        categories = set([item for sublist in categories for item in sublist])
        product_diversity[rec_name] = len(categories)
    return product_diversity

# Cost Diversity

In [83]:
# We will use the "Price" column in the items catalog to determine cost diversity in the recommendations.
# The recommender with a high standard deviation in the cost across all its recommendations is said to be cost-diverse
def get_cost_diversity(user_id, n):
    top_n = get_top_n(user_id,n)
    cost_diversity = {}
    for rec_name in top_n:
        std_dev = np.std(items.loc[top_n[rec_name]][['Price']].values)
        cost_diversity[rec_name] = std_dev
    return cost_diversity

# Serendipity

In [144]:
# We will use inverse popularity as a measure of serendipity.
# The recommender with least number of recommendations on the "most popular" list, will be called most serendipitous
def get_serendipity(user_id, n):
    top_n = get_top_n(user_id,n)
    popular_items = get_popular_items(20)
    serendipity = {}
    for rec, rec_name in zip(recs, recs_names):
        popularity = np.sum(np.isin(top_n[rec_name],popular_items))
        if int(popularity) == 0:
            serendipity[rec_name] = 1
        else:
            serendipity[rec_name] = 1/popularity
    return serendipity

# Average metrics

In [161]:
avg_metrics = {}
for name in recs_names: 
    avg_metrics[name] = {"rmse": [], "precision_at_n": [], "product_diversity": [], "cost_diversity": [], "serendipity": []}

for user_id in ratings.columns:
    if user_id == 'item':
        continue
    user_id = str(user_id)
    rmse = get_rmse(user_id)
    precision_at_n = get_precision_at_n(user_id, 10)
    product_diversity = get_product_diversity(user_id, 10)
    cost_diversity = get_cost_diversity(user_id, 10)
    serendipity = get_serendipity(user_id, 10)
    for key in avg_metrics:
        rec_name = avg_metrics[key]
        rec_name['rmse'].append(rmse[key])
        rec_name['precision_at_n'].append(precision_at_n[key])
        rec_name['product_diversity'].append(product_diversity[key])
        rec_name['cost_diversity'].append(cost_diversity[key])
        rec_name['serendipity'].append(serendipity[key])

# The Price for certain items is not available. Also rmse for certain users is turning out to be NaN.
# Ignoring nans in the average metric calculation for now. So basically narrowing down the evaluation to users who have
# rated atleast one item and items for which the price is known.
for key in avg_metrics:
    rec_name = avg_metrics[key]
    for metric in rec_name:
        temp = rec_name[metric]
        temp = [x for x in temp if not np.isnan(x)]
        rec_name[metric] = sum(temp) / len(temp)

In [162]:
avg_metrics

{'cbf': {'rmse': 0.5723872810703614,
  'precision_at_n': 0.06099999999999995,
  'product_diversity': 9.16,
  'cost_diversity': 21.001285934775364,
  'serendipity': 1.0},
 'item_item': {'rmse': 0.574672000873145,
  'precision_at_n': 0.073,
  'product_diversity': 9.29,
  'cost_diversity': 28.369414026218912,
  'serendipity': 1.0},
 'user_user': {'rmse': 0.5451297974933205,
  'precision_at_n': 0.06799999999999996,
  'product_diversity': 9.09,
  'cost_diversity': 26.861774672518923,
  'serendipity': 1.0},
 'pers_bias': {'rmse': 0.6662727799419501,
  'precision_at_n': 0.07499999999999996,
  'product_diversity': 10.0,
  'cost_diversity': 5.4132984399532225,
  'serendipity': 1.0},
 'mf': {'rmse': 0.6590287674772085,
  'precision_at_n': 0.08199999999999996,
  'product_diversity': 9.23,
  'cost_diversity': 38.535530124652226,
  'serendipity': 1.0}}